In [3]:
from numpy import array, hstack
from keras.models import Sequential
from keras.layers import Dense

# Function to split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # Find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # Check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # Gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# Define a new multivariate dataset
in_seq1 = array([100, 200, 300, 400, 500, 600, 700, 800, 900])
in_seq2 = array([150, 250, 350, 450, 550, 650, 750, 850, 950])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])
# Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# Choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# Convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# Flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
# Flatten output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

# Display the input-output pairs
print("\nInput-Output Pairs:")
print("| Input (X)                                            | Output (y)                   |")
print("|------------------------------------------------------|------------------------------|")
for i in range(len(X)):
    input_values = X[i].reshape((n_steps_in, 3))
    output_values = y[i].reshape((n_steps_out, 3))
    print(f"| {input_values.tolist()} | {output_values.tolist()} |")

# Define the model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=2000, verbose=0)

# Demonstrate prediction with new input
x_input = array([[600, 650, 1250], [700, 750, 1450], [800, 850, 1650]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)

# Display the prediction
print("\nPrediction:")
print(f"Given input sequence {x_input.flatten().tolist()}, the predicted output is:")
predicted_values = yhat[0].reshape((n_steps_out, 3))
for step, prediction in enumerate(predicted_values, 1):
    print(f"Step {step}: {prediction.tolist()}")


Input-Output Pairs:
| Input (X)                                            | Output (y)                   |
|------------------------------------------------------|------------------------------|
| [[100, 150, 250], [200, 250, 450], [300, 350, 650]] | [[400, 450, 850], [500, 550, 1050]] |
| [[200, 250, 450], [300, 350, 650], [400, 450, 850]] | [[500, 550, 1050], [600, 650, 1250]] |
| [[300, 350, 650], [400, 450, 850], [500, 550, 1050]] | [[600, 650, 1250], [700, 750, 1450]] |
| [[400, 450, 850], [500, 550, 1050], [600, 650, 1250]] | [[700, 750, 1450], [800, 850, 1650]] |
| [[500, 550, 1050], [600, 650, 1250], [700, 750, 1450]] | [[800, 850, 1650], [900, 950, 1850]] |

Prediction:
Given input sequence [600, 650, 1250, 700, 750, 1450, 800, 850, 1650], the predicted output is:
Step 1: [918.9298706054688, 948.3926391601562, 1879.7791748046875]
Step 2: [1025.9324951171875, 1073.69580078125, 2107.171630859375]
